# D&A 컨퍼런스

## First  :  Crawling

In [1]:
import selenium 
from selenium import webdriver
import random
from time import sleep
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm

In [2]:
# Browser 켜기
browser = webdriver.Chrome()
browser.get('https://everytime.kr/timetable')

In [2]:
# login 함수 생성
def login(id_,password):
    browser.find_element_by_name('userid').send_keys(str(id_))
    browser.find_element_by_name('password').send_keys(str(password))
    browser.find_element_by_xpath('//*[@id="container"]/form/p[3]/input').click()

In [6]:
login('kcg99kr','kk1350')

In [8]:
# 가끔 팝업뜨는거 닫아주기
browser.find_element_by_xpath('//*[@id="sheet"]/ul/li[3]/a').click()

In [17]:
browser.find_element_by_xpath('//*[@id="container"]/ul/li[1]').click()

In [18]:
# 스크롤 위치 찾기
scr1=browser.find_element_by_xpath('//*[@id="subjects"]/div[2]')

# 스크롤 계속 내리기 ( 매우 오래걸림 실행 X)
# 하다가 창보고 알아서 끊으면됨
for i in range(1000):
    browser.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scr1)
    sleep(3)

KeyboardInterrupt: 

In [60]:
# 에타 과목 정보 저장
tables = browser.find_elements_by_tag_name("tr")
# 이 과목 평점 저장
stars = browser.find_elements_by_class_name('star')

### 에브리타임 과목정보 크롤링

In [34]:
from tqdm import tqdm_notebook

In [21]:
# 점수랑 타이틀 가져오는 코드
scores=[]
links=[]
for table in tqdm (stars) :
    scores.append(table.get_attribute('title'))
    links.append(table.get_attribute('href'))

100%|██████████████████████████████████████████████████████████████████████████████| 2907/2907 [01:44<00:00, 27.71it/s]


In [71]:
# 테이블이, 3번행부터 정보가 있고, 2918행 이후로는 의미없는 행이길래 제외 (이거 자주 바뀜 확인해줘야 할듯)
# 테이블에서 세부정보 뽑는 코드

tables=tables[3:2909]
grades=[] # 추천학년
categories=[] # 수업 분류(전공,교양)
codes=[] # 과목코드
names=[] # 수업 이름
profs=[] # 교수 이름
weights=[] # 학점(1,2,3)
times=[] # 시간
competitiors=[] # 에타 시간표에 담은인원

In [72]:
for element in tqdm_notebook(tables):
    splited=element.text.split(' ')
    grades.append(splited[0])
    categories.append(splited[1])
    codes.append(splited[2])
    names.append(splited[3])
    profs.append(splited[4])
    weights.append(splited[5])
    try :
        times.append(splited[8]+splited[9]+splited[10]+splited[11]+splited[12])
    except :
        times.append(None)
    try :
        competitiors.append(splited[13])
    except :
        competitiors.append(None)

In [73]:
# 뽑은 리스트들을 df로 바꿔줬음
dfs=[names,codes,scores,categories,profs,times,grades,weights,competitiors,links]
df=pd.DataFrame()
for cols in dfs:
    df=pd.concat([df,pd.Series(cols)],axis=1)

In [74]:
# 칼럼 이름 만들어줌
df.columns=['class_name','class_code','score','category','professor','time',
            'recommend_year','weight','competitor','link']

In [75]:
# 아까 위에서 형식 제대로 안지켜진 데이터들 떨굼 ( 보완 방향 (!) )
df.dropna(inplace=True)

In [76]:
# 교수이름이 3글자 아닌 수업들 뽑아봄 : 왜냐면 그냥 인덱싱으로 한거라 형식 안지켜진건데 들어갔을 수 있음
error_list=[]
for i in df['professor'].unique():
    if len(i) !=3 :
        error_list.append(i)

# 직접 보고 이름 2,4글자인 교수님들 이름 빼줌
non_error=['박진','이화','박민','호티롱안','나강','문영','전긍','고댕','김예솔란','정철',
                 '최웅','안준','김강','이건','사공석진','허림','오정','이민','이찬',
                 '김명','허균','김영','이웅','임강','최영']

# 제거
for i in non_error :
    error_list.remove(i)

In [3]:
# 그냥 이름3자 아니거나, non error 아닌 거 다 없애버렸는데 이거 보완해야함 ( 1 )
df2=df.query('professor not in @error_list') 

# Save
df2.to_csv('df2.csv',index=False)
# Load
#df2=pd.read_csv('df2.csv')

In [4]:
# 링크 추출함, 리뷰 하나도 없는 수업은 java스크립트로 뽑혀오길래 그것도 제외
link_list=list(df2['link'].unique())
link_list.remove('javascript: alert("%EC%95%84%EC%A7%81 %EB%93%B1%EB%A1%9D%EB%90%9C %EA%B0%95%EC%9D%98%ED%8F%89%EC%9D%B4 %EC%97%86%EC%8A%B5%EB%8B%88%EB%8B%A4.");')

### 에브리타임 리뷰크롤링

In [8]:
from random import randint
from bs4 import BeautifulSoup as bs
import requests
import lxml.html
import re
from tqdm import tqdm_notebook

In [6]:
def review_crawl(link_text):
    ## Selenium
    browser.get(link_text)
    sleep(0.5) # 로딩 기다리기
    # 리뷰 내용만 추출
    reg=re.compile('[0-9]')
    reviews={}
    sleep(2) # 로딩 기다리기 
    tb=browser.find_element_by_class_name('articles')
    art_table=tb.find_elements_by_tag_name('article')

    for element in art_table :
        reviews[element.find_element_by_class_name('text').text]=(float(''.join(reg.findall(element.find_element_by_class_name('on').get_attribute('style'))))*0.05)

    # 소스 뺴와서 파싱
    table=bs(browser.page_source,'xml')
    heads=table.find('div',class_='side head')
    
    # 제목, 교수 내용 추출
    class_name=(heads.find('h2').text)
    prof_name=(heads.find('span').text)

    # 강의평 테이블 추출
    articles=table.find('div',class_='side article')

    # 강의평의 평균 평점 내용 추출
    mean_score=float(articles.find('span',class_='value').text)

    # 강의평 중 세부 테이블 추출
    details=articles.find('div',class_='details')

    # 강의평 중 세부 내용 추출
    labels=details.find_all('label')
    detail=details.find_all('span')
    detail_type={}
    for l,d in zip(labels,detail):
        detail_type[str(l.text)]=d.text

    review_t=pd.DataFrame.from_dict(reviews, orient='index').reset_index()
    review_t.columns=['review','score']
    review_t['name']=class_name
    review_t['prof']=prof_name
    
    review_detail=pd.DataFrame.from_dict(detail_type,orient='index').T
    review_detail['name']=class_name
    review_detail['prof']=prof_name

    
    return review_t,review_detail

In [9]:
review=pd.DataFrame() # 강의평 추출 테이블
detail=pd.DataFrame() # 과목 세부정보 추출 테이블

# ↓ 가끔 안 켜고 시작하면 오류뜰 때가 있어서 그때 대비 
browser = webdriver.Chrome()
browser.get(link_list[0]) 
login('kcg99kr','kk1350')
# ↑ 가끔 안 켜고 시작하면 오류뜰 때가 있어서 그때 대비 


# 크롤링 과정 (오래걸림 실행 ㄴ)
for link in tqdm_notebook(link_list) :
    try:
        r_t,r_d=review_crawl(link)
        review=pd.concat([review,r_t])
        detail=pd.concat([detail,r_d])
        sleep(1)
    except:
        print(link , '강의평 추출 X ')

https://everytime.kr/lecture/view/550049 강의평 추출 X 
https://everytime.kr/lecture/view/861357 강의평 추출 X 
https://everytime.kr/lecture/view/549879 강의평 추출 X 
https://everytime.kr/lecture/view/1703566 강의평 추출 X 
https://everytime.kr/lecture/view/554824 강의평 추출 X 
https://everytime.kr/lecture/view/861378 강의평 추출 X 
https://everytime.kr/lecture/view/545973 강의평 추출 X 
https://everytime.kr/lecture/view/545979 강의평 추출 X 
https://everytime.kr/lecture/view/1870547 강의평 추출 X 
https://everytime.kr/lecture/view/1950783 강의평 추출 X 
https://everytime.kr/lecture/view/1084644 강의평 추출 X 
https://everytime.kr/lecture/view/1951399 강의평 추출 X 
https://everytime.kr/lecture/view/1416026 강의평 추출 X 
https://everytime.kr/lecture/view/1870555 강의평 추출 X 
https://everytime.kr/lecture/view/550276 강의평 추출 X 
https://everytime.kr/lecture/view/1950800 강의평 추출 X 
https://everytime.kr/lecture/view/1084895 강의평 추출 X 
https://everytime.kr/lecture/view/541701 강의평 추출 X 
https://everytime.kr/lecture/view/541703 강의평 추출 X 
https://everytime.kr/l

https://everytime.kr/lecture/view/1704180 강의평 추출 X 
https://everytime.kr/lecture/view/1870770 강의평 추출 X 
https://everytime.kr/lecture/view/547575 강의평 추출 X 
https://everytime.kr/lecture/view/1418167 강의평 추출 X 
https://everytime.kr/lecture/view/547579 강의평 추출 X 
https://everytime.kr/lecture/view/1418170 강의평 추출 X 
https://everytime.kr/lecture/view/1788903 강의평 추출 X 
https://everytime.kr/lecture/view/1870775 강의평 추출 X 
https://everytime.kr/lecture/view/1870777 강의평 추출 X 
https://everytime.kr/lecture/view/1870778 강의평 추출 X 
https://everytime.kr/lecture/view/1870780 강의평 추출 X 
https://everytime.kr/lecture/view/859607 강의평 추출 X 
https://everytime.kr/lecture/view/1703853 강의평 추출 X 
https://everytime.kr/lecture/view/1788929 강의평 추출 X 
https://everytime.kr/lecture/view/1870789 강의평 추출 X 
https://everytime.kr/lecture/view/1870792 강의평 추출 X 
https://everytime.kr/lecture/view/1870791 강의평 추출 X 
https://everytime.kr/lecture/view/1418243 강의평 추출 X 
https://everytime.kr/lecture/view/1870793 강의평 추출 X 
https://everyti

https://everytime.kr/lecture/view/1870937 강의평 추출 X 
https://everytime.kr/lecture/view/1870939 강의평 추출 X 
https://everytime.kr/lecture/view/860225 강의평 추출 X 
https://everytime.kr/lecture/view/1704000 강의평 추출 X 
https://everytime.kr/lecture/view/1870942 강의평 추출 X 
https://everytime.kr/lecture/view/1870943 강의평 추출 X 
https://everytime.kr/lecture/view/1704007 강의평 추출 X 
https://everytime.kr/lecture/view/1870944 강의평 추출 X 
https://everytime.kr/lecture/view/860285 강의평 추출 X 
https://everytime.kr/lecture/view/1419047 강의평 추출 X 
https://everytime.kr/lecture/view/860258 강의평 추출 X 
https://everytime.kr/lecture/view/860288 강의평 추출 X 
https://everytime.kr/lecture/view/1870949 강의평 추출 X 
https://everytime.kr/lecture/view/1704022 강의평 추출 X 
https://everytime.kr/lecture/view/1870950 강의평 추출 X 
https://everytime.kr/lecture/view/1704031 강의평 추출 X 
https://everytime.kr/lecture/view/860318 강의평 추출 X 
https://everytime.kr/lecture/view/860355 강의평 추출 X 
https://everytime.kr/lecture/view/1871154 강의평 추출 X 
https://everytime.

https://everytime.kr/lecture/view/1951316 강의평 추출 X 
https://everytime.kr/lecture/view/1951317 강의평 추출 X 
https://everytime.kr/lecture/view/1629506 강의평 추출 X 
https://everytime.kr/lecture/view/1871091 강의평 추출 X 
https://everytime.kr/lecture/view/549104 강의평 추출 X 
https://everytime.kr/lecture/view/1704195 강의평 추출 X 
https://everytime.kr/lecture/view/549112 강의평 추출 X 
https://everytime.kr/lecture/view/544924 강의평 추출 X 
https://everytime.kr/lecture/view/1704226 강의평 추출 X 
https://everytime.kr/lecture/view/553879 강의평 추출 X 
https://everytime.kr/lecture/view/545099 강의평 추출 X 
https://everytime.kr/lecture/view/1871085 강의평 추출 X 
https://everytime.kr/lecture/view/549091 강의평 추출 X 
https://everytime.kr/lecture/view/1871088 강의평 추출 X 
https://everytime.kr/lecture/view/861072 강의평 추출 X 
https://everytime.kr/lecture/view/1704232 강의평 추출 X 
https://everytime.kr/lecture/view/549643 강의평 추출 X 
https://everytime.kr/lecture/view/549646 강의평 추출 X 
https://everytime.kr/lecture/view/861146 강의평 추출 X 
https://everytime.kr/l

In [10]:
#Save
#review.to_csv('review.csv',index=False)
#detail.to_csv('detail.csv',index=False)

In [11]:
#Load
review=pd.read_csv('review.csv')
review=review.reset_index().drop('index',axis=1)

####  현재 가지고 있는 테이블 
테이블 1: 에브리타임 과목정보  
테이블 2: 에브리타임 리뷰  
테이블 3 :에브리타임 리뷰 요약    

In [12]:
review.head()

,review,score,name,prof
0,재미있지만 나에겐 어려웠지 .. 하지만 겨수님 강의방식이 너뮤좋음,3.0,역사란무엇인가,윤은주
1,강의력은 좋으심 그러나 너무 암기위주라 좀 아쉬웠다,2.0,역사란무엇인가,윤은주
2,수업 내용이 많아서 머리아프긴 해도 재밌는 수업\n약술형 서술형을 잘 써낼 정도로 ...,5.0,역사란무엇인가,윤은주
3,"간혹 정치성향을 들어내시지만, 사람은 참 좋으십니다. 시험 문제 중 객관식은 모두 ...",5.0,역사란무엇인가,윤은주
4,교수님 열정있고 최고에요 외울거많고 꽉채워서 강의하셔서 쉴시간없습니다,4.0,역사란무엇인가,윤은주


In [13]:
def cleanText(readData):
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', ' ', readData) # 특문제거
    text = re.sub('[^가-힣]',' ',text) # 한글 제외, + 단일 자음모음 제거 
    return text

In [14]:
clean=[]
for data in review['review']:
    clean.append(cleanText(data))

In [15]:
review['review']=pd.Series(clean)

----------------------

## Second  :  Text-Mining

## Tokenize 

In [17]:
from konlpy.tag import *
kkma = Kkma()

 https://m.blog.naver.com/PostView.nhnblogId=wideeyed&logNo=221337575742&proxyReferer=https%3A%2F%2Fwww.google.com%2F

In [18]:
# 토크나이징하고, 에러난 row ( 대부분 빈 텍스트를 토크나이징 한 것 ) 찾아줌
corpus=[] 
error=[]
i=0
for data in tqdm(review['review']):
    try:
        corpus.append(kkma.morphs(data))
    except:
        error.append(i)
    i += 1

100%|████████████████████████████████████████████████████████████████████████████| 11666/11666 [12:10<00:00, 15.96it/s]


In [939]:
#error=[2568,2720,6009,6532,6589,7729,8722,10853]

In [19]:
error

[2589, 2743, 6065, 6582, 6641, 7787, 8785, 10943]

In [20]:
# 에러난 친구들
display(review.loc[error,:])
review=review.drop(error,0) # 빈 행 삭제

,review,score,name,prof
2589,...,5.0,한국사회와문화,유양석
2743,,2.0,생활속의스포츠,김의진
6065,,1.0,교육학개론,이기종
6582,,5.0,사제동행세미나,이종민
6641,,5.0,광고와사회적이슈,고한준
7787,,1.0,창업연계공학설계입문,김석명
8785,,5.0,컴퓨터구조,임은진
10943,,3.0,동역학,이동헌


In [22]:
import pickle

In [23]:
# save
with open('kkma_token_0923.pickle', 'wb') as f:
    pickle.dump(corpus, f, pickle.HIGHEST_PROTOCOL)

In [24]:
#load
import pickle
with open('kkma_token_0923.pickle', 'rb') as f:
    corpus = pickle.load(f)

_______________ 

## Embedding

In [25]:
from gensim.models import Word2Vec, FastText
embd_SG= Word2Vec(corpus, size=100, window=2, min_count=10,
                           workers=-1, iter=1000, sg=1)
#CBOW보단 Skip - Gram의 성능이 전반적으로 낫다고들 함
#리뷰 여러개니까, 차원 적으면 표현이 안될거라고 생각, 100차원정도 임시로 줘봤음
#리뷰다보니까 별로 안 길어서 윈도우는 작게주고, min_count는 의미없는 단어 나오지 말라고 10정도로 줌
# -> 이거 나중에 보완 ( 2 )

In [27]:
# 리뷰 단위별 벡터구하기 (받은 코드가 옛날버전인듯 word2vec 모델에 그런인자가 없던데..그래서 그냥 짬)
def get_embedding_vector(model,corpus):
    embeding_dict={}
    embeding_df=pd.DataFrame()
    for w,v in zip(model.wv.index2word , model.wv.vectors):
        embeding_dict[w]=v
        embeding_table=(pd.DataFrame(embeding_dict).T.reset_index())
    for words in (corpus):
        embeding_df=pd.concat([embeding_df,
                               embeding_table.query('@words in index').iloc[:,1:].mean()]
                             ,axis=1 )
    return embeding_df

In [28]:
%%time
#임베딩 벡터 추출
embedding_vect=get_embedding_vector(embd_SG,corpus)

C:\Users\kcg99\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


Wall time: 5min 24s


In [29]:
vect=(embedding_vect.T.reset_index()).iloc[:,1:] # 데이터 프레임으로 바꿔주고
vect.columns=['Review_Embeded'+str(x) for x in range(100)] # 칼럼이름도 달아줌

# 리뷰랑 병합(Group화 해서 강의별 계산을 위함)
review.index=range(len(review))
df3=pd.concat([review.iloc[:,1:],vect],axis=1)

# 여기다 담아놓은다음에 Merge
feature=[]
for col_name in (['Review_Embeded'+str(x) for x in range(100)]):
    feature.append(df3.groupby(['name','prof'])[col_name].mean().reset_index())

In [31]:
#Merge의 과정
df=df2.drop(['class_code','time','link','competitor'],axis=1)
df.rename({'class_name':'name','professor':'prof'},axis=1,inplace=True)

for f in feature :
    df =pd.merge(df, f, on=['name','prof'],how='left')

df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

In [33]:
# Skip-Gram을 이용한 인베딩 테이블 완성
sim_df=pd.concat([df[['name','prof',]],df.iloc[:,6:]],axis=1)
sim_df.index=[x for x in range(893)]

# 나중에 사용할 테이블
detail_table=df2[['class_name','professor','category','recommend_year','weight','score']].drop_duplicates()
detail_table.rename({'class_name':'name','professor':'prof'},axis=1,inplace=True)

In [ ]:
"""
df3=pd.concat([df3,pd.get_dummies(df3['category'])],axis=1)
df3=pd.concat([df3,pd.get_dummies(df3['recommend_year'])],axis=1)

df3.drop(['category','recommend_year'],axis=1,inplace=True)

df3=pd.concat([df3[['name','prof','score']],df3.iloc[:,3:]],axis=1)
df3['weight']=df3['weight'].astype(int)

"""


## Similarlity

In [34]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

#코사인 유사도 계산 함수
def cos_sim(A, B): 
       return dot(A, B)/(norm(A)*norm(B))

#다른 모든 행과 유사도 계산 함수
def find_similar(input_name,input_prof,df):
    input_lecture=df.query('name == @input_name & prof == @input_prof')
    rest_lecture=df.query('name != @input_name | prof != @input_prof')
    output=rest_lecture.iloc[:,:2]
    output['similar']=''
    for i in range(len(rest_lecture)):
        output.iloc[i,2]= cos_sim(input_lecture.iloc[:,2:],rest_lecture.iloc[i,2:])
    output = pd.merge(output, detail_table, on=['name','prof'],how='left')
    output = output.sort_values('similar',ascending=False)
    return output

In [35]:
# 내가 들었던 수업중에 교수님이 (사람)좋았던 수업
find_similar('문명과세계화의도전','안현상',sim_df)[:5] # 경제학개론 확인

,name,prof,similar,category,recommend_year,weight,score
48,미디어와공동체,김성준,[0.9096037601182206],핵심교양,1-4,3,4.05
16,시민정신과사회봉사,여현철,[0.9064828464110245],핵심교양,1-4,3,4.19
89,과학기술과인간사고의변화,이근세,[0.905356076280653],핵심교양,1-4,3,4.14
736,경제학개론,유은나,[0.8994474962401916],전공선택,1,3,4.68
204,교육사회학,윤옥한,[0.8979353515877092],교직,2,2,4.20


In [36]:
# 강의는 썩 재미없었지만 교수님이 열정 넘치고 엄청 좋은 분이었음, 유은나 교수님의 경제학 개론도 대체적으로 비슷한 평가
display(review.query('name=="문명과세계화의도전" & prof =="안현상"')[['review']][:5])
display(review.query('name=="경제학개론" & prof =="유은나"')[['review']][:5]) 

,review
2589,외우기 자신 있고 세계사나 이런거 조금 좋아하면 정말 개꿀일듯 주시는 프린트만...
2590,수업자료 나올거 같은것들 외으면 됨 성적 잘주시는듯 약술형시험인데 그냥 프린트 그...
2591,세계사의 굵직한 흐름들을 스토리텔링으로 알려주셔서 쉽게 맥을 잡을 수 있다 중간 ...
2592,갓갓이라는 말이 맞음 과제업고 조별로 활동하는것도 없고 한학기내내 교수님강의와 ...
2593,암기에 자신있으신 분들 무조건 들으시길 시험 하루전날에 죽고싶은거 빼면 정말 ...


,review
9490,유은나 교수님 정말 짱아십니다 강의 전달력은 진심 모든 교수님 중에 최고이신 듯 수...
9491,이 교수님을 배정으로 받으셨다면 박수먼저 드립니다 교수님 성격 좋으시고 알려주시는...
9492,정말정말 강의력 최고십니다 또 교수님께 경제학 배우고 싶어요 다만 과제는 굉장히 귀...
9493,교수님 최고입니다 자기소개는 조금 의문이었지만 좋아요
9494,강제배정으로 들었는데 강제배정해준 학교에게 고마웠던 수업 강의 듣기 전에 경제에 ...


In [37]:
# 외우기만 해서 맘에 안들었던 수업, 영화시리즈 제외하고 한문과 문화 확인
display(find_similar('영화작문','이지현',sim_df)[:5]) 

,name,prof,similar,category,recommend_year,weight,score
107,영화독해,임미진,[0.8963967854572485],자유교양,1-4,2,3.10
108,영화독해,김혜정,[0.8899280611361691],자유교양,1-4,2,3.68
112,영화듣기,노윤아,[0.8839941530175607],자유교양,1-4,2,3.80
47,논리와소통,조종화,[0.8835795971795675],핵심교양,1-4,3,3.43
79,세계역사의이해,윤은주,[0.8811142969402759],핵심교양,1-4,3,4.26


In [38]:
# 암기라는 키워드 자주등장, 차이점은 교수님 칭찬이 많음 ㅎ; 평점차인듯
display(review.query('name=="영화작문" & prof =="이지현"')[['review']][:5])
display(review.query('name=="논리와소통" & prof =="조종화"')[['review']][:5]) 


,review
3674,수업시간에 강조하신 문장들을 철저히 암기만 한다면 영어울렁증이 있어도 에이플 가능합...
3675,암기가 중요한 수업입니다 그러나 영화 자체를 반복해서 자주 보고 대사 위주 약간의...
3676,자기가 영어를 잘한다 암기에 문제 없다 그러면 무조건 받아갑니다 년 내...
3677,외국인반 들었는데 엄청 쉬웠어요 한국어를 못해도 시험 잘 볼 듯 명중에서 ...
3678,생각보다 학점따기 어렵습니다 열심히 하는 사람도 많구요 암기 싫어하시는 분 비...


,review
1585,피피티만 외우면 에이플 가능해요 어렵지 않고 내용 쉬워요 대신 필기할게 많아서 빠...
1586,수업 빠지면 치명적 아침 시간은 힘들듯
1587,출석 부르다 안 부르다 하시는데 출튀 많은 날은 출석 다시 부르십니다 전체적으로 ...
1588,버거운 전공과목들 사이에 한줄기 빛같은 교수님 애초에 수업을 시험문제 갖고 ...
1589,등록금 버리고 싶으면 들으세요 저는 진짜 논리에대한건 하나도 배워간게 없는듯합니다 ...


## Pre-Trained Embedding

In [39]:
from gensim.models import FastText

In [ ]:
%%time
model = FastText.load_fasttext_format('D:/fasttext/wiki.ko.bin')
# Pre-triainde vector를 사용해보고 싶은데, 커뮤니티 리뷰다 보니까 별 이상한 단어랑 줄임말,오타들이 다있음
# -> 그래서 일반적 word2vec모델에선 사용이 안되것 같음
#   -> 근데 FastText는 corpus에 없는 단어들도 사용 가능 + Noise에 강한모델
#      -> 따라서 FastText를 이용한 Pre_Trained Vector 사용 시도

C:\Users\kcg99\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  """Entry point for launching an IPython kernel.


In [54]:
fatx_vec=[] # 문장별 임베딩
for words in corpus:
    fatx_vec.append(model[words].mean(axis=0))

C:\Users\kcg99\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [55]:
fatx_emd=pd.DataFrame() # 임베딩한 벡터를 데이터프레임으로 만들어줌
for vecs in (fatx_vec):
    fatx_emd=pd.concat([fatx_emd,pd.DataFrame(vecs)],axis=1)

#벡터 데이터프레임 전처리    
fatx_emd=fatx_emd.T
fatx_emd.index=range(len(fatx_emd))
fatx_emd.columns=['Review_ft_Embeded'+str(x) for x in range(300)]

#그룹바이 하기위해 달아줌
fatx_df=pd.concat([review[['name','prof']],fatx_emd],axis=1)

In [57]:
# 강의별 임베딩 평균값 생성
feature=[]
for col_name in (['Review_ft_Embeded'+str(x) for x in range(300)]):
    feature.append(fatx_df.groupby(['name','prof'])[col_name].mean().reset_index())

In [58]:
# 중복값 제거
fatx_df=fatx_df.iloc[:,:2].drop_duplicates()

In [59]:
# 평균값과 강의명 병합
for f in feature :
    fatx_df =pd.merge(fatx_df, f, on=['name','prof'],how='left')

In [60]:
# 임베딩 테이블 완성
fatx_df.dropna(inplace=True)

In [61]:
display(find_similar('문명과세계화의도전','안현상',fatx_df)[:5])
display(find_similar('문명과세계화의도전','안현상',sim_df)[:5])

display(find_similar('영화작문','이지현',fatx_df)[:5])
display(find_similar('영화작문','이지현',sim_df)[:5])
# 결과가 다름 : 수업 분야가 비슷한것들끼리 조금더 잘 뽑은 느낌 (내 주관 + 내가 알고있는 평가)
# 가중치를 줘서 앙상블하는 것도 괜찮지 않을까?

,name,prof,similar,category,recommend_year,weight,score
735,경제학개론,유은나,[0.9988283144631275],전공선택,1,3,4.67
80,세계역사의이해,고재백,[0.998548050556071],핵심교양,1-4,3,4.34
123,맑시즘의이해,김경래,[0.9978231463359574],자유교양,1-4,2,4.14
16,시민정신과사회봉사,여현철,[0.9977997396624142],핵심교양,1-4,3,4.19
119,동아시아근대사와인물,류미나,[0.9977124303865662],자유교양,1-4,2,3.60


,name,prof,similar,category,recommend_year,weight,score
738,경제학개론,유은나,[0.9296383760575444],전공선택,1,3,4.67
16,시민정신과사회봉사,여현철,[0.9253792534243485],핵심교양,1-4,3,4.19
48,미디어와공동체,김성준,[0.901716117828681],핵심교양,1-4,3,4.02
902,글쓰기(공학인증),노영근,[0.8949482251001085],기초교양,1,3,3.70
80,세계역사의이해,고재백,[0.8926219292275788],핵심교양,1-4,3,4.34


,name,prof,similar,category,recommend_year,weight,score
108,영화독해,임미진,[0.998631855646347],자유교양,1-4,2,3.08
113,영화듣기,노윤아,[0.9981571576633029],자유교양,1-4,2,3.82
109,영화독해,김혜정,[0.9978252653663491],자유교양,1-4,2,3.67
58,시민사회와정치,김학량,[0.9977450449617042],핵심교양,1-4,3,3.48
119,동아시아근대사와인물,류미나,[0.9977199706578582],자유교양,1-4,2,3.60


,name,prof,similar,category,recommend_year,weight,score
113,영화듣기,노윤아,[0.9394146710621524],자유교양,1-4,2,3.82
108,영화독해,임미진,[0.9330671486226814],자유교양,1-4,2,3.08
79,세계역사의이해,윤은주,[0.9165696466232842],핵심교양,1-4,3,4.30
109,영화독해,김혜정,[0.9130228554471296],자유교양,1-4,2,3.67
105,영상을통한영어,서지영,[0.8992512418012921],자유교양,1-4,2,4.23


### 개선방안
 1) 강의평은 학생이 주관적으로 쓰는것, 게다가 온라인커뮤니티상이라 대충쓰는 경우가 많으며, 에타는 리뷰가 적어서 그게 잘 걸러지지도 않음  
   > 수업계획서를 크롤링해서, 내용이 비슷한것과, 교수님 성향이 비슷한 걸 나눠줄 수 있지 않을까?  
   
   
 2) 머신러닝 모델링을 진행 안했음
   > 이를 이용해 평점에 어떤게 가장 영향을 많이 미치는 지 알 수 있었을텐데 ( 교수 피드백용으로 사용 할 수 있지 않을까?)
   
 3) 디테일 테이블이랑 병합을 할 수 있음
   > 이를 통해 유사도 말고도, 다양한 방법으로 필터링 할 수 있을텐데, 구현해야할 듯

관련 링크:  
 1) https://ratsgo.github.io/natural%20language%20processing/2017/03/08/word2vec/  
 2) https://fasttext.cc/docs/en/pretrained-vectors.html  
 3) https://byeongkijeong.github.io/fastText/  
 4) https://ratsgo.github.io/from%20frequency%20to%20semantics/2017/03/11/embedding/
 
 1) word2vec의 전반적 사용법    
 2) fasttext pre-trained-vectors 구한곳  
 3) 그거 사용법 찾은곳   
 4) 임베딩의 전반적 정보 얻은곳

In [ ]:
"""### 전처리 해보려고 했으나 과정이 너무 번거로워서 나중에 Develop 

error_index=df.query('professor in @error_list').index #

error_index.astype(int)

error_table=[]
for i in error_index:
    error_table.append(tables[i].text)

#### 에러의 유형 :
1) 팀팀class의 제목명이 길어서 오류 ( 팀팀class에 대한 분석은 우리task에 별로 맞지 않는듯 ) -> 제거    
2) 강의이름이 영어라 강의 제목에 띄어쓰기가 들어감  
3) 외국인 교수라 교수 이름란에 띄어쓰기가 들어감

import re
error_team= re.compile('팀팀Class')
error_english = re.compile('[a-zA-Z]')

#### 팀팀삭제

error_table2=[]
for table in error_table :
    if len(error_team.findall(table)) == 0 :
        error_table2.append(table)

#### 영어제목 수정

error_eng=[]
error_table3=[]
for table in error_table2:
    if len(error_english.findall(table)) < 5:
        error_table3.append(table)
    else :
        error_eng.append(table)

error_eng[8]

reg_kor=re.compile('[가-힣]')

eng_title=[]
for element in error_eng:
    valid_chr=''.join(element.split(' ')[3:])
    point=(reg_kor.search(valid_chr)).start()
    if len(valid_chr[:point]) >= 1:
        eng_title.append(valid_chr[:point])
    else 

eng_title

### 전처리 해보려고 했으나 과정이 너무 번거로워서 나중에 Develop 
"""